In [1]:
!pip install -q polars


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


c:\Users\32mou\OneDrive\바탕 화면\adfadsfaf\snapshot\defillama


In [ ]:
import polars as pl
from datetime import datetime
from util import *

# Web Crawling

In [1]:
min_date = []
max_date = []
files = os.listdir("result")
len(files)

4539

In [2]:
for d in files:
    df = pl.read_csv(f"result/{d}").sort("date")
    if df.height > 2 and df[-1, "date"] == df[-2, "date"]:
        df = df[:-1]
    df = cast_datetime(df)
    df = truncate_datetime(df, trun="1m")

    temp = df["date"].value_counts().sort("date").filter(pl.col("count") > 1)
    if temp.shape[0]:
        print(d, "has duplicates")
        print(temp)

    df = df.unique("date", keep="last")
    min_date.append(df["date"].dt.min())
    max_date.append(df["date"].dt.max())


min_date = pl.Series("min_date", min_date).dt.min()
max_date = pl.Series("max_date", max_date).dt.max()

print(min_date, max_date)

result_raw = get_empty_result(
    min_date, max_date, col="date")

debridge.csv has duplicates
shape: (7, 2)
┌─────────────────────┬───────┐
│ date                ┆ count │
│ ---                 ┆ ---   │
│ datetime[μs]        ┆ u32   │
╞═════════════════════╪═══════╡
│ 2022-02-25 21:00:00 ┆ 2     │
│ 2022-02-26 12:00:00 ┆ 2     │
│ 2022-02-28 16:00:00 ┆ 2     │
│ 2022-03-01 08:00:00 ┆ 2     │
│ 2022-07-06 20:59:00 ┆ 2     │
│ 2022-07-12 21:00:00 ┆ 2     │
│ 2022-07-13 05:03:00 ┆ 2     │
└─────────────────────┴───────┘
fantom.csv has duplicates
shape: (5, 2)
┌─────────────────────┬───────┐
│ date                ┆ count │
│ ---                 ┆ ---   │
│ datetime[μs]        ┆ u32   │
╞═════════════════════╪═══════╡
│ 2021-03-28 10:02:00 ┆ 2     │
│ 2021-05-22 21:42:00 ┆ 2     │
│ 2021-05-22 23:36:00 ┆ 2     │
│ 2021-05-23 01:06:00 ┆ 2     │
│ 2021-06-26 05:57:00 ┆ 2     │
└─────────────────────┴───────┘
2018-04-27 09:00:00 2024-08-18 16:39:00


In [10]:
STEP = 500
os.listdir("result")

for i in range(0, len(files), STEP):
    result = result_raw
    for d in files[i:i+STEP]:
        df = pl.read_csv(f"result/{d}").sort("date")
        if df.height > 2 and df[-1, "date"] == df[-2, "date"]:
            df = df[:-1]
        df = cast_column_type_without_date(df)
        df = cast_datetime(df, has_ms=False, col="date")
        df = truncate_datetime(df, "date", "1m")
        df = leave_first_last(df, d[:-4])

        result = result.join(df, on="date", how="left")

    result = drop_null_rows(result)
    result = result.sort("date")
    result.write_csv(f"result{STEP}/result{i}.csv")

# 1 hour step

In [11]:
import os
import polars as pl
from util import change_hourly, cast_column_type_without_date

def hourly_df(path):
    df = cast_datetime(pl.read_csv(path), has_ms=True)
    df = cast_column_type_without_date(df, float)

    df = df.with_columns([
        pl.col('date').dt.truncate('1h').alias('hourly_date')])

    hourly_df = change_hourly(df)

    return hourly_df.sort('hourly_date').drop(['date'])

In [17]:
STEP = 2000
os.makedirs(f"result{STEP}", exist_ok=True)

for start in range(0, len(files), STEP):
    df1 = hourly_df(f"result500/result{start}.csv")
    df2 = hourly_df(f"result500/result{start+500}.csv")
    result = df1.join(df2, on="hourly_date", how="full", coalesce=True)
    
    if os.path.isfile(f"result500/result{start+1000}.csv"):
        df3 = hourly_df(f"result500/result{start+1000}.csv")
        result = result.join(df3, on="hourly_date", how="full", coalesce=True)
    if os.path.isfile(f"result500/result{start+1500}.csv"):
        df4 = hourly_df(f"result500/result{start+1500}.csv")     
        result = result.join(df4, on="hourly_date", how="full", coalesce=True)
        
    result.write_csv(f"result{STEP}/result{start}.csv")

c:\Users\32mou\OneDrive\바탕 화면\adfadsfaf\snapshot\defillama\util.py:46: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col(col).str.to_datetime(
